In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import fitparse as fp
import csv
import pytz
import datetime
from pathlib import Path
import os
import shutil
from pathlib import Path
import re

In [2]:
def print_record(data):
    """
    test method to print out the data in a record
    """
    for record in data:
        for field in record:
            if field.units:
                print (" * %s: %s %s" % (
                    field.name, field.value, field.units))
            else:
                print (" * %s: %s" % (field.name, field.value))

In [3]:
def get_time(record):
    """
    extract the time from data. Note for heart rate data the timestamp is int(16) up to 65536 ~ 18hours
    This will overflow and needs normalised so it wont give a crazy result on overflow.

    Every now and then there will be a timestamp giving the right date and time. 
    Heart rate data is timestamped on a rolling seconds counter. Need to set the first timestamp_16 to the
    global time then work out the next time based on the difference between timestamp_16s adding that to global time
    """
    new_timestamp_16 = None
    
    # difference between successive timestamp_16
    delta_diff = 0
    global timestamp_16
    global current_timestamp

    for r in record:
        if r.field is not None:
            if r.name == 'timestamp_16':
                new_timestamp_16 = r.value

    # if the timestamp is none then set it to the global time
    if new_timestamp_16 is not None:
        if timestamp_16 is None:
            timestamp_16 = new_timestamp_16
        
        delta_diff = new_timestamp_16 - timestamp_16
        
        # deal with overflow
        if delta_diff < 0:
            delta_diff += 65536
        
        current_timestamp += datetime.timedelta(seconds=delta_diff)
        timestamp_16 = new_timestamp_16
        #print(current_timestamp)
        return current_timestamp

In [4]:
def get_heartrate(record):
    """
    Return the heart rate from the record
    """
    for r in record:
        if r.name == 'heart_rate':
            return r.value

In [5]:
def output_messages(fitfile):
    """
    Go through the fitfile and extract time and heart rate data
    """
    messages = fitfile.get_messages()

    current_timestamp = datetime.datetime.now()
    timestamp_16 = None

    hrdata = []
    timestamp = []

    for record in messages:
    # Go through all the data entries in this record

        # Extract the global time stamp
        if record.name == 'monitoring_info':
            for r in record.fields:
                if r.field is not None:
                    if r.field.name == 'local_timestamp':
                        current_timestamp = r.value
                        timestamp_16 = None
                        #print("Timestamp: {}".format(current_timestamp))

        # get the heart rate data
        if record.name == 'monitoring':
            for r in record:
                if r.field is not None:
                    if r.field.name =='heart_rate':
                        time = get_time(record)
                        hr = get_heartrate(record)

                        timestamp.append(time)
                        hrdata.append(hr)
                        #print("Time: {}   Heart: {}".format(time,hr))

In [6]:
def output_csv(time, hr, fi):
    """
    Write the extracted time and heart rate data to csv fiile
    """
    data = [time, hr]

    csvFile = open(fi,'w')
    with csvFile:
        print("Writing csv to: {}".format(fi))
        writer = csv.writer(csvFile, lineterminator='\n') # need lineterminator to prevent extra linebreaks
        writer.writerow(["Date time", "Heart rate BPM"])
        for i in range(len(time)):
            writer.writerow([time[i],hr[i]])

In [7]:

def copy_files_without_email(source, destination):
    r = []
    for root, dirs, files in os.walk(source):
        for name in files:
            filepath = root + os.sep + name
            if filepath.endswith(".fit"):
                name = Path(filepath)
                #print(name.stem)
                no_email = re.findall(r'(?<=_).*$', name.stem) # look for anything after _
                #print(no_email[0])
                if len(no_email)  is not None:
                    new_name = no_email[0] + name.suffix
                    dest = destination + new_name
                    #print(dest)
                    shutil.copy(filepath, destination+new_name)
                

In [22]:
raw_data_dir = Path(r"Garmin_data/data")
dest_data_dir = Path(r"Garmin_data/fit/")

In [ ]:
#copy_files_without_email(raw_data_dir, dest_data_dir)

In [34]:
#os.listdir(dest_data_dir)

In [92]:
#grab last 4 characters of the file name:
def filenumber_beginning(x):
    # get file name (xxx.fit) and split on .fit
    # if filename has xxx_yyy.fit then split on _yyy.fit
    # return xxx
    filename = os.path.splitext(x)[0]
    
    if '_' in filename:
        filename = filename.split('_')[0]

    file_out = int(filename)

    return(file_out)

file_list = os.listdir(dest_data_dir)

s = sorted(file_list, key = filenumber_beginning) 

print(f"files {len(s)}")

files 8523


In [97]:
parse_file = dest_data_dir.joinpath(s[100])
fitfile = fp.FitFile(str(parse_file)) # needs string file path and not windows path that sorted produces

In [98]:
di = fitfile.get_messages('device_info')
for r in di:
    for x in r.fields:
        if x.name =="garmin_product":
            print(x.value)

fr310xt_4t
1080
fr310xt_4t
1080
fr310xt_4t
1080
fr310xt_4t
1080
fr310xt_4t
1080
fr310xt_4t
1080
fr310xt_4t
1080
fr310xt_4t
1080


In [ ]:
# for f in os.listdir(dest_data_dir):

#     filename = os.path.splitext(f)[0]
    
#     if '_' in filename:
#         filename = filename.split('_')[0]

#     filename = int(filename)
#     print(filename)

In [12]:
# # get the fit files in order (maybe)
# fit_files_sorted = sorted(Path(dest_data_dir).glob('*.fit'))

In [ ]:
# fit_files_sorted[0].__str__()

In [ ]:
# fitfile = fp.FitFile(fit_files_sorted[-1].__str__()) # needs string file path and not windows path that sorted produces
# fitfile

In [41]:
mesx = fitfile.get_messages()
mesx

<generator object FitFile.get_messages at 0x0000022E69F79B40>

In [42]:
for r in mesx:
    print(f"-{r}")
    for x in r:
        print(f"    {x}")

-file_id (#0)
    garmin_product: fr310xt_4t
    manufacturer: garmin
    number: None
    serial_number: 3871167223
    time_created: 2016-05-26 18:01:47
    type: activity
-file_creator (#49)
    hardware_version: None
    software_version: 450
-event (#21)
    event: timer
    event_group: 0
    event_type: start
    timer_trigger: manual
    timestamp: 2016-05-26 18:01:46
-device_info (#23)
    battery_status: None
    battery_voltage: None [V]
    cum_operating_time: None [s]
    device_index: creator
    device_type: 1
    garmin_product: fr310xt_4t
    hardware_version: None
    manufacturer: garmin
    serial_number: 3871167223
    software_version: 4.5
    timestamp: 2016-05-26 18:01:46
    unknown_15: None
    unknown_16: None
    unknown_8: None
    unknown_9: None
-device_info (#23)
    battery_status: None
    battery_voltage: None [V]
    cum_operating_time: None [s]
    device_index: 1
    device_type: 12
    garmin_product: 1080
    hardware_version: None
    manufactur

look for device_info-> garmin_product.
The fit files will change depending on what device it is 

In [49]:
for r in mesx:
    print(f"-{r.name}")
    #for x in r:
        #print(f"    {x}")

In [ ]:
for f in fit_files_sorted:
    try:
        print(f"converting {f.name}")
        fitfile = fp.FitFile(f.__str__())
        
        output_messages(fitfile)
        
    except fp.utils.FitCRCError as err:
        print(type(err))    # the exception type
        print(err.args)     # arguments stored in .args
        print(err) 
        print(f"error with {f.name}")
        
        
#fi = datadir+'/'+'op.csv'
#output_csv(timestamp, hrdata, fi)

In [ ]:
#choose the right directory here fit files are located
dir = "data/Fit_data/"
testdir = "C:/temp/data/test"
datadir = dir

UTC = pytz.UTC
GMT = pytz.timezone('Europe/London')

# heart rate data list
hrdata = []
# corresponding timestamp
timestamp = []

# globals to get the time data
current_timestamp = datetime.datetime.now()
timestamp_16 = None

In [ ]:
# get the fit files in order (maybe)
files = sorted(Path(datadir).glob('*.fit'))

for f in files:

    ff = datadir+'/'+f.

    fitfile = fitparse.FitFile(ff)

    print("converting {}".format(f.name))

    output_messages(fitfile)

fi = datadir+'/'+'op.csv'
output_csv(timestamp, hrdata, fi)


In [ ]:
np.uint16(np.iinfo(np.uint16).max) # unit16 max

In [ ]:
blah = [1,2,3,4,5,6]
blah[-2:]